In [ ]:
#Setting up the system

In [ ]:
#basic libraries

In [ ]:
import os 
import zipfile 
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model 
import matplotlib.pyplot as plt

In [ ]:
os.getcwd()

In [ ]:
os.chdir('''MyWorkingDirectory''')

In [ ]:
base_dir = os.getcwd()

train_dir = 'HW5train'
val_dir = 'HW5val'
test_dir = 'HW5test'

train_cats_dir = os.path.join(base_dir, train_dir, 'cat')
train_dogs_dir = os.path.join(base_dir, train_dir, 'dog')
train_elephants_dir = os.path.join(base_dir, train_dir, 'elephant')

val_cats_dir = os.path.join(base_dir, val_dir, 'cat')
val_dogs_dir = os.path.join(base_dir, val_dir, 'dog')
val_elephants_dir = os.path.join(base_dir, val_dir, 'elephant')

test_cats_dir = os.path.join(base_dir, test_dir, 'cat')
test_dogs_dir = os.path.join(base_dir, test_dir, 'dog')
test_elephants_dir = os.path.join(base_dir, test_dir, 'elephant')

In [ ]:
b_s = 10#fp #batch_size
s_p_e = 9#sp #steps per epoch
n_e = 20#tp #number of epochs
#experience says that having a very large number of epochs is a bad idea

In [ ]:
#Check whether images have been loaded correctly
import matplotlib.image as mpimg
nrows = 6
ncols = 4

fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index = 100

#get the names of the individual files
train_cat_fnames = os.listdir( train_cats_dir )
train_dog_fnames = os.listdir( train_dogs_dir )
train_elephant_fnames = os.listdir( train_elephants_dir )

#grab eight pictures from each file
next_cat_pix = [os.path.join(train_cats_dir, fname) 
                for fname in train_cat_fnames[ pic_index-8:pic_index] 
               ]

next_dog_pix = [os.path.join(train_dogs_dir, fname) 
                for fname in train_dog_fnames[ pic_index-8:pic_index]
               ]

next_elephant_pix = [os.path.join(train_elephants_dir, fname) 
                for fname in train_elephant_fnames[ pic_index-8:pic_index]
               ]

#put them is a grid to look at.
for i, img_path in enumerate(next_cat_pix+next_dog_pix+next_elephant_pix):
  # Set up subplot; subplot indices start at 1
  sp = plt.subplot(nrows, ncols, i + 1)
  sp.axis('Off') # Don't show axes (or gridlines)

  img = mpimg.imread(img_path)
  plt.imshow(img)

plt.show()

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1.0/255. )
val_datagen = ImageDataGenerator( rescale = 1.0/255. )
test_datagen = ImageDataGenerator( rescale = 1.0/255. )

In [ ]:
train_generator = train_datagen.flow_from_directory(train_dir, batch_size = b_s, class_mode = 'categorical', target_size = (150, 150))
validation_generator = test_datagen.flow_from_directory(val_dir, batch_size = b_s, class_mode = 'categorical', target_size = (150, 150))

In [ ]:
#Load the base model
#i.e. download it from https://storage.googleapis.com/tesorflow/keras-applications/inception_v3/inception_v3_weights_tf_dim_ordering_tf_kerels_notop.h5
from tensorflow.keras.applications.inception_v3 import InceptionV3
from keras import models
from keras import layers
model = models.Sequential()
base_model = InceptionV3(input_shape = (150, 150, 3), include_top = False, weights = 'imagenet')
model.add(base_model)

In [ ]:
#Make sure that we don't train the weights.
for layer in base_model.layers:
    layer.trainable = False

In [ ]:
#Create the model to go on top of the base model
from tensorflow.keras.optimizers import Adamax
import keras

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(3, activation='softmax'))

lr = 4 

opt = Adamax(learning_rate=10**-lr)
model.compile(optimizer=opt
              ,loss='categorical_crossentropy'
              ,metrics=['accuracy'])

In [ ]:
#Let's see a summary
print(model.summary())

In [ ]:
#fit the model
import time
start = time.time()

history = model.fit(
    train_generator
    , validation_data = validation_generator
    , steps_per_epoch = s_p_e
    , epochs = n_e)

end = time.time()
print('Number of seconds: ',end - start)

In [ ]:
#plotting loss and accuracy
history_dict = history.history
#print(history_dict.keys())
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
# "bo" is for "blue dot"
import matplotlib.pyplot as plt
plt.plot(epochs[1:len(epochs)], loss[1:len(loss)], 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs[1:len(epochs)], val_loss[1:len(val_loss)], 'b', label='Validation loss')
plt.axes().set_xticks([0,2,4,6,8,10,12,14,16,18,20])
plt.title('Training and validation loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()
plt.clf()

In [ ]:
plt.plot(epochs, acc, 'ro', label = 'Training Accuracy')
plt.plot(epochs, val_acc, 'r', label = 'Validation Accuracy')
plt.axes().set_xticks([0,2,4,6,8,10,12,14,16,18,20])
plt.title('Training and validation accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
plt.clf()

In [ ]:
import operator
diff = list(map(operator.sub, acc, val_acc))
diff2 = list(map(operator.sub, loss, val_loss))
plt.plot(epochs, diff, 'r', label = 'Accuracy Difference')
plt.plot(epochs, diff2, 'b', label = 'Loss Difference')
plt.axhline(y = 0, color = 'g', linestyle = '--') 
plt.axes().set_xticks([0,2,4,6,8,10,12,14,16,18,20])
plt.title('Training minus validation accuracy difference')
plt.xlabel('Epoch')
plt.ylabel('Accuracy Difference')
plt.legend()
plt.show()
plt.clf()